# Python3入门到精通——Python 套接字编程

作者： Daniel Meng

GitHub： [LibertyDream](https://github.com/LibertyDream)

博客：[明月轩](https://libertydream.github.io/)

> 本系列教程采用[知识共享署名-非商业性使用-相同方式共享 2.5 中国大陆许可协议](http://creativecommons.org/licenses/by-nc-sa/2.5/cn/)

要实现网络编程，需要回顾一下网络结构，以 TCP/IP 五层模型为例

<img src="https://raw.githubusercontent.com/LibertyDream/diy_img_host/master/img/2020-02-24_tcp_ip_5_layers.png" style="zoom:50%;" />

| OSI层      | 功能                         | TCP/IP协议                     |
| ---------- | ---------------------------- | ------------------------------ |
| 应用层     | 文件传输、电子邮件、文件服务 | HTTP、FTP、SMTP、DNS、Telnet等 |
| 传输层     | 提供端对端的接口             | TCP、UDP                       |
| 网络层     | 为数据包选择路由             | IP、ICMP等                     |
| 数据链路层 | 传输有地址的帧、错误检测功能 | ARP等                          |
| 物理层     | 物理媒体                     | 1000BASE-SX等                  |

http 是浏览器常用协议，TCP 则是传输层代表。客户端请求报文自上而下封装，传输到服务端再自下而上解包，处理响应。应用层协议和具体应用高度耦合，不适于开发使用，所以系统会提供套接字 socket 将 TCP 等下层网络结构封装，供我们自定义网络传输实现方式

一般套接字程序流程如下

![](https://raw.githubusercontent.com/LibertyDream/diy_img_host/master/img/2020-02-24_socket_programming.gif)

Python 中对应套接字模块为 `socket`，写个例子看一下

In [1]:
import socket
''' server.py'''
server = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
server.bind(('0.0.0.0', 8000))  # 绑定地址与端口
server.listen()
sock, addr = server.accept()
data = sock.recv(1024)  # 一次接受 1KB 数据
print(data.decode('utf8'))
sock.send(('Hello %s' % data.decode('utf8')).encode('utf8'))
sock.close()
server.close()

In [ ]:
import socket
''' client.py'''
client = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
client.connect(('127.0.0.1', 8000))
client.send('Daniel Meng'.encode('utf8'))
data = client.recv(1024)
print(data.decode('utf8'))
client.close()

建立套接字首先需要传入参数协议族，常用的有 AF_INET（ipv4），AF_INET6（ipv6），AP_IPX（linux进程间通信）。第二个参数指定协议，SOCK_STREAM 代表 TCP，SOCK_DGRAM 代表 UDP

如果要应对多用户，且要求会话由通信双方沟通决定是否断开，可以简单轮询

In [ ]:
import socket
import threading
''' server_multi_client.py'''
server = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
server.bind(('0.0.0.0', 8000))  # 绑定地址与端口
server.listen()

def handle_sock(sock, addr):
    while True:
        data = sock.recv(1024)  # 一次接受 1KB 数据
        accept_str = data.decode('utf8')
        if accept_str == 'exit':
            break
        else:
            print(accept_str)
        message = input()
        sock.send(message.encode('utf8'))
    sock.send('disconnect'.encode('utf8'))
    sock.close()


while True:
    sock, addr = server.accept()
    client_threading = threading.Thread(target=handle_sock, args=(sock, addr))
    client_threading.start()

In [ ]:
import socket
''' multi_client.py'''
client = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
client.connect(('127.0.0.1', 8000))
client.send('Connect'.encode('utf8'))
while True: 
    message = input()
    client.send(message.encode('utf8'))
    data = client.recv(1024)
    accept_str = data.decode('utf8')
    if accept_str == 'disconnect':
        break
    print(accept_str)
print('disconnect')
client.close()

套接字底层灵活但编码量也大，很多基于不同网络协议的框架基于在 socket 上做了高层封装，比如 urllib， requests。也可以自己模拟一下高层协议的通信，下面以 http 访问百度为例

In [5]:
import socket
from urllib.parse import urlparse

def get_url(url):

    url_p = urlparse(url)
    host = url_p.netloc
    path = url_p.path
    if path == "":
        path = "/"

    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client.connect((host,80))

    client.send("GET {} HTTP/1.1\r\nHost:{}\r\nConnection:close\r\n\r\n".format(path, host).encode('utf8'))

    data = b""
    while True:
        d = client.recv(1024)
        if d:
            data += d
        else:
            break

    data = data.decode('utf8').split('\r\n\r\n')[1]
    print(data)
    client.close()

In [6]:
get_url("http://www.baidu.com")

<!DOCTYPE html><!--STATUS OK-->
<html>
<head>
	<meta http-equiv="content-type" content="text/html;charset=utf-8">
	<meta http-equiv="X-UA-Compatible" content="IE=Edge">
	<link rel="dns-prefetch" href="//s1.bdstatic.com"/>
	<link rel="dns-prefetch" href="//t1.baidu.com"/>
	<link rel="dns-prefetch" href="//t2.baidu.com"/>
	<link rel="dns-prefetch" href="//t3.baidu.com"/>
	<link rel="dns-prefetch" href="//t10.baidu.com"/>
	<link rel="dns-prefetch" href="//t11.baidu.com"/>
	<link rel="dns-prefetch" href="//t12.baidu.com"/>
	<link rel="dns-prefetch" href="//b1.bdstatic.com"/>
	<title>百度一下，你就知道</title>
	<link href="http://s1.bdstatic.com/r/www/cache/static/home/css/index.css" rel="stylesheet" type="text/css" />
	<!--[if lte IE 8]><style index="index" >#content{height:480px\9}#m{top:260px\9}</style><![endif]-->
	<!--[if IE 8]><style index="index" >#u1 a.mnav,#u1 a.mnav:visited{font-family:simsun}</style><![endif]-->
	<script>var hashMatch = document.location.href.match(/#+(.*wd=[^&].+)/);if (